In [52]:
import os
import csv
from io import StringIO
from datetime import datetime
import numpy as np
import pandas as pd

# ============================
# FUNÇÕES AUXILIARES (ETL)
# ============================

def carregar_secoes_csv(conteudo_bytes: bytes) -> dict:
    """
    Lê um CSV em texto com seções iniciadas por '##NomeSecao'
    e devolve um dicionário {nome_secao: [linhas_csv]}
    """
    texto = conteudo_bytes.decode("utf-8-sig")
    linhas = texto.splitlines()

    secoes = {}
    secao_atual = None
    buffer = []

    for linha in linhas:
        linha = linha.strip()
        if linha.startswith("##"):
            if secao_atual and buffer:
                secoes[secao_atual] = buffer
                buffer = []
            secao_atual = linha.strip("#")
        elif linha:
            buffer.append(linha)

    if secao_atual and buffer:
        secoes[secao_atual] = buffer

    return secoes


def criar_dataframe(dados: list[str]) -> pd.DataFrame:
    """
    Recebe uma lista de linhas (strings) e converte em DataFrame.
    Supõe primeira linha como header.
    """
    if not dados:
        return pd.DataFrame()

    leitor = csv.reader(StringIO("\n".join(dados)))
    linhas = list(leitor)

    if len(linhas) < 2:
        return pd.DataFrame(columns=linhas[0]) if linhas else pd.DataFrame()

    return pd.DataFrame(linhas[1:], columns=linhas[0])


def tratar_decimal(df: pd.DataFrame, col: str) -> pd.Series:
    return pd.to_numeric(
        df[col].astype(str).str.replace(",", ".", regex=False),
        errors="coerce"
    )


def tratar_data_hora(df: pd.DataFrame, col_data: str) -> pd.DataFrame:
    df[col_data] = pd.to_datetime(df[col_data], errors="coerce")
    df["data"] = df[col_data].dt.date
    df["hora"] = df[col_data].dt.time
    return df


def processar_arquivo(conteudo_bytes: bytes):
    """
    Executa TODO o ETL usando o conteúdo do arquivo carregado.
    Retorna os 4 dataframes: combustivel, despesas, servicos, rotas
    """
    secoes = carregar_secoes_csv(conteudo_bytes)

    df_combustivel = criar_dataframe(secoes.get("Refuelling", []))
    df_despesas    = criar_dataframe(secoes.get("Expense", []))
    df_servicos    = criar_dataframe(secoes.get("Service", []))
    df_rotas       = criar_dataframe(secoes.get("Route", []))

    # ============================
    # TRATAMENTO - DESPESAS
    # ============================
    if not df_despesas.empty:
        df_despesas["Odômetro (km)"] = tratar_decimal(df_despesas, "Odômetro (km)")
        df_despesas["Valor total"]   = tratar_decimal(df_despesas, "Valor total")
        df_despesas["Local da despesa"] = df_despesas["Local da despesa"].str.split(" / ").str[0]

        df_despesas = tratar_data_hora(df_despesas, "Data")

        df_despesas.rename(
            columns={
                "Odômetro (km)": "odometro",
                "Data": "data_hora",
                "Valor total": "custo",
                "Tipo de despesa": "tipo_despesa",
                "Local da despesa": "local",
                "Motorista": "motorista",
                "Forma de pagamento": "forma_pagamento",
                "Observação": "observacoes",
            },
            inplace=True,
        )

        df_despesas = df_despesas.astype(
            {
                "odometro": float,
                "custo": float,
                "tipo_despesa": str,
                "local": str,
                "motorista": str,
                "forma_pagamento": str,
                "observacoes": str,
            }
        )

    # ============================
    # TRATAMENTO - TRAJETOS (ROTAS)
    # ============================
    if not df_rotas.empty:
        df_rotas["Odômetro inicial"] = tratar_decimal(df_rotas, "Odômetro inicial")
        df_rotas["Odômetro final"]   = tratar_decimal(df_rotas, "Odômetro final")
        df_rotas["Odômetro (km)"]     = tratar_decimal(df_rotas, "Odômetro (km)")

        df_rotas["origem_tipo"] = df_rotas["Origem"].astype(str).str[-1]
        df_rotas["Origem"] = df_rotas["Origem"].str.split(" / ").str[0]
        df_rotas["destino_tipo"] = df_rotas["Destino"].astype(str).str[-1]
        df_rotas["Destino"] = df_rotas["Destino"].str.split(" / ").str[0]

        df_rotas["tipo_rota"] = df_rotas.apply(
            lambda row: "Urbano"
            if row["Origem"] == row["Destino"]
            else ("Urbano" if row["origem_tipo"] == "U" and row["destino_tipo"] == "U" else "Rodoviario"),
            axis=1,
        )

        df_rotas.rename(
            columns={
                "Data inicial": "data_inicio",
                "Data final": "data_fim",
                "Odômetro inicial": "odometro_inicial",
                "Odômetro final": "odometro_final",
                "Odômetro (km)": "odometro",
                "Valor km": "custo_km",
                "Total": "custo",
                "Origem": "origem",
                "Destino": "destino",
                "Motivo": "motivo",
                "Motorista": "motorista",
                "Observação": "observacoes",
            },
            inplace=True,
        )

        df_rotas["data_inicio"] = pd.to_datetime(df_rotas["data_inicio"], errors="coerce")
        df_rotas["data_fim"]    = pd.to_datetime(df_rotas["data_fim"], errors="coerce")
        df_rotas["tipo_rota"]   = df_rotas["tipo_rota"].astype("category")

    # ============================
    # TRATAMENTO - COMBUSTÍVEL
    # ============================
    if not df_combustivel.empty:
        df_combustivel.columns = [f"{col}_{i}" for i, col in enumerate(df_combustivel.columns)]
        df_combustivel.drop(df_combustivel.columns[7:17], axis=1, inplace=True)

        df_combustivel.rename(
            columns={
                "Odômetro (km)_0": "odometro",
                "Data_1": "data_hora",
                "Combustível_2": "combustivel",
                "Preço / L_3": "custo_por_litro",
                "Valor total_4": "custo",
                "Volume_5": "volume_abastecido",
                "Completou o tanque_6": "tanque_completo",
                "Média_17": "media",
                "Distância_18": "distancia_percorrida",
                "Posto de combustível_19": "posto",
                "Motorista_20": "motorista",
                "Motivo_21": "motivo",
                "Forma de pagamento_22": "forma_pagamento",
                "Observação_23": "observacoes",
            },
            inplace=True,
        )

        for c in ["odometro", "custo_por_litro", "custo", "volume_abastecido", "distancia_percorrida"]:
            if c in df_combustivel.columns:
                df_combustivel[c] = tratar_decimal(df_combustivel, c)

        if "media" in df_combustivel.columns:
            df_combustivel["media"] = pd.to_numeric(
                df_combustivel["media"]
                .astype(str)
                .str.replace(r"[^\d,.]", "", regex=True)
                .str.replace(",", ".", regex=False),
                errors="coerce",
            )

        if "data_hora" in df_combustivel.columns:
            df_combustivel = tratar_data_hora(df_combustivel, "data_hora")

        # =========================================
        # DISTÂNCIA CALCULADA ENTRE ABASTECIMENTOS
        # =========================================
        df_combustivel = df_combustivel.sort_values(
            by=["data_hora", "odometro"],
            ascending=[True, True],
            na_position="last"
        ).reset_index(drop=True)

        df_combustivel["odometro_proximo"] = df_combustivel["odometro"].shift(-1)
        df_combustivel["distancia_calculada"] = df_combustivel["odometro_proximo"] - df_combustivel["odometro"]
        df_combustivel.loc[df_combustivel["distancia_calculada"] < 0, "distancia_calculada"] = np.nan
        df_combustivel.drop(columns=["odometro_proximo"], inplace=True)

        # =========================================================
        # MÉDIA CALCULADA (km/L) - litros na linha atual, km nas anteriores
        # =========================================================
        df_combustivel = df_combustivel.sort_values(
            by=["data_hora"],
            ascending=True,
            na_position="last"
        ).reset_index(drop=True)

        flag_sim = (
            df_combustivel["tanque_completo"]
            .astype(str).str.strip().str.lower()
            .eq("sim")
        )

        segmento = flag_sim.cumsum()

        km_por_segmento = df_combustivel.groupby(segmento)["distancia_calculada"].sum(min_count=1)

        litros_nao_por_segmento = (
            df_combustivel.loc[~flag_sim]
            .groupby(segmento)["volume_abastecido"]
            .sum(min_count=1)
        )

        idx_sim = df_combustivel.index[flag_sim]
        seg_atual = segmento.loc[idx_sim]
        seg_base = seg_atual - 1

        km_base = km_por_segmento.reindex(seg_base).to_numpy()

        litros_base_nao = (
            litros_nao_por_segmento.reindex(seg_base)
            .fillna(0)
            .to_numpy()
        )

        litros_atual = df_combustivel.loc[idx_sim, "volume_abastecido"].to_numpy()
        litros_total = litros_base_nao + litros_atual

        media = km_base / litros_total

        df_combustivel["media_calculada"] = np.nan
        df_combustivel.loc[idx_sim, "media_calculada"] = media

        df_combustivel.loc[
            df_combustivel["media_calculada"].replace([np.inf, -np.inf], np.nan).isna(),
            "media_calculada"
        ] = np.nan
        df_combustivel.loc[df_combustivel["media_calculada"] < 0, "media_calculada"] = np.nan
        df_combustivel.loc[idx_sim[litros_total <= 0], "media_calculada"] = np.nan

        df_combustivel["media_calculada_preenchida"] = df_combustivel["media_calculada"].bfill()

        # =========================================================
        # MÉDIA ACUMULADA (km/L) - regra do "carro zero km tanque cheio"
        #
        # - Não calcula na 1ª linha
        # - Começa na 2ª linha: soma volume (linha0+linha1) e distância (linha0+linha1)
        # - Continua acumulando até o "primeiro registro mais recente" com tanque_completo == "Não"
        # =========================================================
        vol = pd.to_numeric(df_combustivel["volume_abastecido"], errors="coerce")
        dist = pd.to_numeric(df_combustivel["distancia_calculada"], errors="coerce")

        # acumulados "brutos" (cumsum ignora NaN por padrão; NaN vira "não soma")
        vol_cum = vol.cumsum()
        dist_cum = dist.cumsum()

        # não calcula na 1ª linha
        df_combustivel["volume_abastecido_acumulado"] = np.nan
        df_combustivel["distancia_calculada_acumulada"] = np.nan
        df_combustivel["media_acumulada"] = np.nan

        if len(df_combustivel) >= 2:
            df_combustivel.loc[1:, "volume_abastecido_acumulado"] = vol_cum.iloc[1:].to_numpy()
            df_combustivel.loc[1:, "distancia_calculada_acumulada"] = dist_cum.iloc[1:].to_numpy()

            denom = df_combustivel.loc[1:, "volume_abastecido_acumulado"].replace({0: np.nan})
            df_combustivel.loc[1:, "media_acumulada"] = (
                df_combustivel.loc[1:, "distancia_calculada_acumulada"] / denom
            )

        # Para de acumular no "primeiro registro mais recente" com tanque_completo == "Não"
        # (ou seja: remove a média acumulada na linha do "Não" e em todas as posteriores)
        flag_nao = (
            df_combustivel["tanque_completo"]
            .astype(str).str.strip().str.lower()
            .isin(["não", "nao"])
        )

        if flag_nao.any():
            stop_idx = flag_nao[flag_nao].index.max()  # mais recente (maior índice)
            df_combustivel.loc[
                stop_idx:,
                ["volume_abastecido_acumulado", "distancia_calculada_acumulada", "media_acumulada"]
            ] = np.nan


    # ============================
    # TRATAMENTO - SERVIÇOS (se existir)
    # ============================

    return df_combustivel, df_despesas, df_servicos, df_rotas


In [53]:
# ============================
# EXEMPLO DE USO (caminho)
# ============================
caminho = 'data/BD.csv'
df_combustivel, df_despesas, df_servicos, df_rotas = processar_arquivo_por_caminho(caminho)

In [54]:
df_combustivel

,odometro,data_hora,combustivel,custo_por_litro,custo,volume_abastecido,tanque_completo,media,distancia_percorrida,posto,...,forma_pagamento,observacoes,data,hora,distancia_calculada,media_calculada,media_calculada_preenchida,volume_abastecido_acumulado,distancia_calculada_acumulada,media_acumulada
0,1.0,2024-11-04 16:12:12,Gas. Comum,0.01,0.60,60.000,Sim,0.00,187.0,Shell,...,Dinheiro,,2024-11-04,16:12:12,187.0,NaN,12.127894,NaN,NaN,NaN
1,188.0,2024-11-07 07:13:34,Gas. Aditivada,6.61,101.92,15.419,Sim,12.13,387.0,Posto Kalu,...,Cartão de crédito,,2024-11-07,07:13:34,387.0,12.127894,12.127894,75.419,574.0,7.610814
2,575.0,2024-11-09 21:03:53,Gas. Comum,6.79,67.90,10.000,Não,12.53,344.0,Posto Kalu,...,Cartão de crédito,,2024-11-09,21:03:53,344.0,NaN,12.525917,85.419,918.0,10.747023
3,919.0,2024-11-10 16:02:28,Gas. Aditivada,6.61,319.65,48.359,Sim,12.53,519.0,Posto Kalu,...,Cartão de crédito,,2024-11-10,16:02:28,519.0,12.525917,12.525917,133.778,1437.0,10.741677
4,1438.0,2024-12-14 08:02:58,Gas. Aditivada,6.61,70.00,10.590,Não,11.16,87.0,Posto Kalu,...,Cartão de crédito,,2024-12-14,08:02:58,87.0,NaN,11.155260,144.368,1524.0,10.556356
5,1525.0,2024-12-14 16:45:14,Gas. Aditivada,6.61,200.00,30.257,Não,11.16,298.0,Posto Kalu,...,Cartão de crédito,,2024-12-14,16:45:14,298.0,NaN,11.155260,174.625,1822.0,10.433787
6,1823.0,2024-12-16 17:30:04,Gas. Aditivada,6.61,265.66,40.191,Sim,11.16,430.0,Posto Kalu,...,Cartão de crédito,,2024-12-16,17:30:04,430.0,11.155260,11.155260,214.816,2252.0,10.483390
7,2253.0,2024-12-21 10:01:45,Gas. Aditivada,6.49,241.86,37.267,Sim,11.54,374.0,Auto Desk DDD,...,Cartão de crédito,,2024-12-21,10:01:45,374.0,11.538358,11.538358,252.083,2626.0,10.417204
8,2627.0,2024-12-21 16:32:53,Gas. Aditivada,6.11,197.89,32.388,Sim,11.55,698.0,Posto Coopervap,...,Cartão de crédito,,2024-12-21,16:32:53,698.0,11.547487,11.547487,284.471,3324.0,11.684847
9,3325.0,2024-12-28 19:50:03,Gas. Comum,6.29,378.96,60.248,Sim,11.59,152.0,Posto Tabari Ltda,...,Cartão de crédito,,2024-12-28,19:50:03,152.0,11.585447,11.585447,344.719,3476.0,10.083575


In [55]:
print(df_combustivel["distancia_calculada"].sum())
print(df_combustivel["distancia_percorrida"].sum())

18437.0
18437.0


In [56]:
df_combustivel.to_excel('data/combustivel.xlsx', index=False)